In [71]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from pathlib import Path
import re

### Loading the data

In [72]:
def import_data(file_path , file_type='auto'):
	try:
		if file_type == 'csv' or file_path.endswith('.csv'):
			return pd.read_csv(file_path)
		else:
			return pd.read_excel(file_path)
	except Exception as e:
		print(f"Error importing {file_path}: {e}")
		raise e
try:
	ad_category_df = pd.read_excel(r'C:\Users\GUMMUDU HEMANTH\Bharat-Herald-Analytics\data_raw\dim_ad_category.xlsx')
	city_df = pd.read_excel(r'C:\Users\GUMMUDU HEMANTH\Bharat-Herald-Analytics\data_raw\dim_city.xlsx')
	ad_revenue_df = pd.read_csv(r'C:\Users\GUMMUDU HEMANTH\Bharat-Herald-Analytics\data_raw\fact_ad_revenue.csv')
	city_readiness_df = pd.read_csv(r'C:\Users\GUMMUDU HEMANTH\Bharat-Herald-Analytics\data_raw\fact_city_readiness.csv')
	digital_plot_df = pd.read_csv(r'C:\Users\GUMMUDU HEMANTH\Bharat-Herald-Analytics\data_raw\fact_digital_pilot.csv')
	print_sales_df = pd.read_excel(r'C:\Users\GUMMUDU HEMANTH\Bharat-Herald-Analytics\data_raw\fact_print_sales.xlsx')

	print("All datasets imported successfully.")

except Exception as e:
	print(f"An error occurred: {e}")
	raise e

All datasets imported successfully.


### Data preprocessing


In [73]:
def data_preprocessing(df, df_name):
	print(f"Preprocessing {df_name}")
	print("\n" + "-"*50 + "\n")

	# Check missing values
	missing_values = df.isnull().sum()
	if missing_values.any():
		print(f"Missing values in {df_name}:{missing_values[missing_values > 0]}")
	else:
		print(f"No missing values in {df_name}.")

	# Check for duplicates
	duplicate_values = df.duplicated().sum()
	if duplicate_values > 0:
		print(f"Found {duplicate_values} duplicate rows in {df_name}.")
	else:
		print(f"No duplicates rows in {df_name}.")

	# Check size of the dataframe
	print(f"{df_name} shape: {df.shape}")

	# Check columns and its data types
	print(f"{df_name} columns and data types:\n{df.dtypes}")

	# Basic statistics for numerical columns
	print(f"Basic statistics for {df_name}:\n{df.describe(include='all')}")
	print("\n" + "="*100 + "\n")


# Preprocess each dataframe
data_preprocessing(ad_category_df, 'ad_category_df')
data_preprocessing(city_df, 'city_df')
data_preprocessing(ad_revenue_df, 'ad_revenue_df')
data_preprocessing(city_readiness_df, 'city_readiness_df')
data_preprocessing(digital_plot_df, 'digital_plot_df')
data_preprocessing(print_sales_df, 'print_sales_df')


Preprocessing ad_category_df

--------------------------------------------------

No missing values in ad_category_df.
No duplicates rows in ad_category_df.
ad_category_df shape: (4, 4)
ad_category_df columns and data types:
ad_category_id          object
standard_ad_category    object
category_group          object
example_brands          object
dtype: object
Basic statistics for ad_category_df:
       ad_category_id standard_ad_category     category_group example_brands
count               4                    4                  4              4
unique              4                    4                  3              4
top              A001           Government  Commercial Brands       LIC, SBI
freq                1                    1                  2              1


Preprocessing city_df

--------------------------------------------------

No missing values in city_df.
No duplicates rows in city_df.
city_df shape: (10, 4)
city_df columns and data types:
city_id    object
city

In [74]:
digital_plot_df.head()

,Unnamed: 0,platform,launch_month,ad_category_id,dev_cost,marketing_cost,users_reached,downloads_or_accesses,avg_bounce_rate,cumulative_feedback_from_customers,city_id
0,0,PDF WhatsApp Push,2021-01,A001,236570,66060,23509,16319,52.55,"Mixed feedback: some usability concerns, but h...",C001
1,1,PDF WhatsApp Push,2021-02,A001,156865,99122,19472,17017,82.53,"Mixed feedback: some usability concerns, but h...",C002
2,2,PDF WhatsApp Push,2021-03,A001,242728,46087,8471,2891,68.06,"Mixed feedback: some usability concerns, but h...",C003
3,3,PDF WhatsApp Push,2021-04,A001,147695,78868,46796,15640,66.17,"Mixed feedback: some usability concerns, but h...",C004
4,4,PDF WhatsApp Push,2021-05,A001,325906,135644,16805,3231,76.90,The site takes too long to load on average pho...,C005


### Handling Missing Values

In [75]:
def handling_missing_values(ad_revenue_df , digital_plot_df):
	# Handling missing values in ad_revenue_df
	ad_revenue_df['comments'] = ad_revenue_df['comments'].fillna('No Commensts')

	# Handling missing values in digital_plot_df
	digital_plot_df['cumulative_feedback_from_customers'] = digital_plot_df['cumulative_feedback_from_customers'].fillna(method='ffill')

	return ad_revenue_df , digital_plot_df

ad_revenue_df , digital_plot_df = handling_missing_values(ad_revenue_df , digital_plot_df)


### Handling Inconsistencies in Ad_Revenue_Df Table

In [76]:
ad_revenue_df.sample(10)

,edition_id,ad_category,quarter,ad_revenue,currency,comments
645,ED1003,A001,4th Qtr 2023,2948726.00,INR,No Commensts
173,ED1007,A002,Q3-2024,1706575.00,INR,No Commensts
418,ED1004,A003,4th Qtr 2024,2938517.00,INR,No Commensts
701,ED1010,A004,Q1-2020,3454438.00,INR,New FMCG client onboarded
154,ED1008,A003,Q1-2022,3344710.00,INR,No Commensts
395,ED1010,A001,4th Qtr 2023,1717472.00,INR,No Commensts
618,ED1005,A001,Q1-2019,36491.02,USD,No Commensts
698,ED1007,A004,4th Qtr 2023,2774254.00,INR,No Commensts
323,ED1007,A003,Q1-2024,38350.06,EUR,No Commensts
169,ED1001,A001,Q3-2020,2333715.00,IN RUPEES,No Commensts


In [77]:
def preprocess_ad_revenue(ad_revenue_df):
	# Create a copy
	ad_revenue_df_copy = ad_revenue_df.copy()

	# Standardize Quarter Column
	ad_revenue_df_copy['year'] = ad_revenue_df_copy['quarter'].str.extract(r'(\d{4})')
	ad_revenue_df_copy['quarter_num'] = ad_revenue_df_copy['quarter'].str.extract(r'([Qq]\d|Qtr\d|[1-4]th\s*Qtr)', flags=re.IGNORECASE)
	ad_revenue_df_copy['quarter_num'] = ad_revenue_df_copy['quarter_num'].str.replace('[^0-9]', '', regex=True)

	# Standardize ad_revenue and currency column
	usd_to_inr = 88.14
	eur_to_inr = 103.39

	ad_revenue_df_copy['currency'] = ad_revenue_df_copy['currency'].str.upper().str.strip()

	# Conditions for each currency type
	conditions = [
		ad_revenue_df_copy['currency'].isin(['INR' , 'IN RUPEES']),
		ad_revenue_df_copy['currency'] == 'USD',
		ad_revenue_df_copy['currency'] == 'EUR'
	]

	# Corresponding choices for each condition
	choices = [
		ad_revenue_df_copy['ad_revenue'],
		ad_revenue_df_copy['ad_revenue'] * usd_to_inr,
		ad_revenue_df_copy['ad_revenue'] * eur_to_inr
	]

	ad_revenue_df_copy['ad_revenue_inr'] = np.select(conditions, choices, default=np.nan)
	ad_revenue_df_copy['ad_revenue_inr'] = ad_revenue_df_copy['ad_revenue_inr'].round(2)
	return ad_revenue_df_copy

ad_revenue_df = preprocess_ad_revenue(ad_revenue_df)


In [78]:
ad_revenue_df.head()

,edition_id,ad_category,quarter,ad_revenue,currency,comments,year,quarter_num,ad_revenue_inr
0,ED1005,A001,2023-Q2,22613.69,EUR,No Commensts,2023,2,2338029.41
1,ED1005,A002,Q1-2019,39366.88,USD,No Commensts,2019,1,3469796.80
2,ED1001,A003,Q3-2023,3709860.00,INR,No Commensts,2023,3,3709860.00
3,ED1003,A002,Q3-2023,40969.55,USD,No Commensts,2023,3,3611056.14
4,ED1007,A003,4th Qtr 2020,51779.40,USD,No Commensts,2020,4,4563836.32


### Handling Inconsistencies in Digital Plot table

In [79]:
digital_plot_df.sample(10)

,Unnamed: 0,platform,launch_month,ad_category_id,dev_cost,marketing_cost,users_reached,downloads_or_accesses,avg_bounce_rate,cumulative_feedback_from_customers,city_id
5,5,PDF WhatsApp Push,2021-06,A001,248972,53225,35268,16551,45.74,"Mixed feedback: some usability concerns, but h...",C006
4,4,PDF WhatsApp Push,2021-05,A001,325906,135644,16805,3231,76.90,The site takes too long to load on average pho...,C005
32,32,Mobile App Beta,2021-09,A002,194914,36256,34309,18913,48.24,"Mixed feedback: some usability concerns, but h...",C003
20,20,E-paper Mobile Web,2021-09,A003,247823,72092,10017,3841,60.59,"Mixed feedback: some usability concerns, but h...",C001
0,0,PDF WhatsApp Push,2021-01,A001,236570,66060,23509,16319,52.55,"Mixed feedback: some usability concerns, but h...",C001
24,24,Mobile App Beta,2021-01,A002,234834,67861,27436,23043,65.63,"Mixed feedback: some usability concerns, but h...",C005
22,22,E-paper Mobile Web,2021-11,A003,85110,96077,34252,18724,67.27,"Mixed feedback: some usability concerns, but h...",C003
40,40,Responsive Web Version,2021-05,A004,172197,63787,47821,30567,83.53,"Mixed feedback: some usability concerns, but h...",C001
29,29,Mobile App Beta,2021-06,A002,187539,87775,48363,30763,79.31,"Mixed feedback: some usability concerns, but h...",C010
30,30,Mobile App Beta,2021-07,A002,98313,35469,31201,26898,50.07,"Mixed feedback: some usability concerns, but h...",C001


In [80]:
def preprocess_digital_plot(digital_plot_df):
	# Create a copy
	digital_plot_df_copy = digital_plot_df.copy()

	# Convert 'launch_month' to datetime
	digital_plot_df_copy['launch_month'] = pd.to_datetime(digital_plot_df_copy['launch_month'], errors='coerce')

	# Extract year , month , month_name and quarter
	digital_plot_df_copy['launch_year'] = digital_plot_df_copy['launch_month'].dt.year
	digital_plot_df_copy['launch_month_num'] = digital_plot_df_copy['launch_month'].dt.month
	digital_plot_df_copy['launch_month_name'] = digital_plot_df_copy['launch_month'].dt.month_name()
	digital_plot_df_copy['launch_quarter'] = digital_plot_df_copy['launch_month'].dt.quarter

	return digital_plot_df_copy

digital_plot_df_test = preprocess_digital_plot(digital_plot_df)

In [81]:
digital_plot_df_test.sample(10)

,Unnamed: 0,platform,launch_month,ad_category_id,dev_cost,marketing_cost,users_reached,downloads_or_accesses,avg_bounce_rate,cumulative_feedback_from_customers,city_id,launch_year,launch_month_num,launch_month_name,launch_quarter
40,40,Responsive Web Version,2021-05-01,A004,172197,63787,47821,30567,83.53,"Mixed feedback: some usability concerns, but h...",C001,2021,5,May,2
2,2,PDF WhatsApp Push,2021-03-01,A001,242728,46087,8471,2891,68.06,"Mixed feedback: some usability concerns, but h...",C003,2021,3,March,1
9,9,PDF WhatsApp Push,2021-10-01,A001,221877,95697,47845,21517,75.39,"Mixed feedback: some usability concerns, but h...",C010,2021,10,October,4
11,11,PDF WhatsApp Push,2021-12-01,A001,106832,44226,37373,28240,44.09,"Mixed feedback: some usability concerns, but h...",C002,2021,12,December,4
22,22,E-paper Mobile Web,2021-11-01,A003,85110,96077,34252,18724,67.27,"Mixed feedback: some usability concerns, but h...",C003,2021,11,November,4
25,25,Mobile App Beta,2021-02-01,A002,415324,120038,22251,7564,77.10,Many said font was too tiny to read.\nZooming ...,C006,2021,2,February,1
14,14,E-paper Mobile Web,2021-03-01,A003,99873,58731,23284,7898,48.29,"Mixed feedback: some usability concerns, but h...",C005,2021,3,March,1
7,7,PDF WhatsApp Push,2021-08-01,A001,188777,66727,39865,17866,76.14,"Mixed feedback: some usability concerns, but h...",C008,2021,8,August,3
38,38,Responsive Web Version,2021-03-01,A004,160073,46184,48619,20364,83.98,"Mixed feedback: some usability concerns, but h...",C009,2021,3,March,1
37,37,Responsive Web Version,2021-02-01,A004,199654,57386,8384,5805,83.89,"Mixed feedback: some usability concerns, but h...",C008,2021,2,February,1


### Handling Inconsistencies in City Readiness Table

In [82]:
city_readiness_df.sample(10)

,Unnamed: 0,city_id,quarter,literacy_rate,smartphone_penetration,internet_penetration
69,69,C003,2024-Q2,82.37,69.32,67.78
171,171,C008,2019-Q4,71.32,79.22,76.65
12,12,C001,2022-Q1,89.18,75.96,55.68
169,169,C008,2019-Q2,71.44,79.13,74.06
85,85,C004,2022-Q2,75.58,68.97,66.27
180,180,C008,2022-Q1,71.29,78.50,75.62
47,47,C002,2024-Q4,70.95,49.13,48.00
89,89,C004,2023-Q2,75.46,69.47,65.45
132,132,C006,2022-Q1,81.73,49.07,74.31
216,216,C010,2019-Q1,70.50,75.62,74.61


In [83]:
def preprocess_city_readiness(city_readiness_df):
	# Create a copy
	city_readiness_df_copy = city_readiness_df.copy()

	# Extract year and quarter_num from Quarter Column
	city_readiness_df_copy['year'] = city_readiness_df_copy['quarter'].str.extract(r'(\d{4})')
	city_readiness_df_copy['quarter_num'] = city_readiness_df_copy['quarter'].str.extract(r'([Qq]\d|Qtr\d|[1-4]th\s*Qtr)', flags=re.IGNORECASE)
	city_readiness_df_copy['quarter_num'] = city_readiness_df_copy['quarter_num'].str.replace('[^0-9]', '', regex=True)

	return city_readiness_df_copy

city_readiness_df = preprocess_city_readiness(city_readiness_df)


In [84]:
city_readiness_df.sample(10)

,Unnamed: 0,city_id,quarter,literacy_rate,smartphone_penetration,internet_penetration,year,quarter_num
224,224,C010,2021-Q1,71.07,76.32,73.51,2021,1
201,201,C009,2021-Q2,74.89,70.78,74.62,2021,2
152,152,C007,2021-Q1,66.11,77.47,63.49,2021,1
22,22,C001,2024-Q3,88.98,74.15,57.03,2024,3
91,91,C004,2023-Q4,75.55,68.49,70.09,2023,4
1,1,C001,2019-Q2,88.76,76.45,55.97,2019,2
160,160,C007,2023-Q1,66.90,77.71,62.93,2023,1
43,43,C002,2023-Q4,70.58,47.84,47.76,2023,4
137,137,C006,2023-Q2,81.95,46.34,74.22,2023,2
235,235,C010,2023-Q4,70.92,77.75,74.83,2023,4


In [85]:
city_readiness_df.isna().sum()

Unnamed: 0                0
city_id                   0
quarter                   0
literacy_rate             0
smartphone_penetration    0
internet_penetration      0
year                      0
quarter_num               0
dtype: int64

### Handling Inconsistencies in Print Sales Table


In [86]:
print_sales_df.head()

,edition_ID,City_ID,Language,State,Month,Copies Sold,copies_returned,Net_Circulation
0,ED1005,C005,Hindi,Rajasthan,2023-05-01 00:00:00,404389,13510,390879
1,ED1005,C005,Hindi,Rajasthan,2019-03-01 00:00:00,492943,25024,467919
2,ED1001,C001,hindi,Uttar pradesh,2023-07-01 00:00:00,168893,12285,156608
3,ED1003,C003,Hindi,Madhya_Pradesh,2023-07-01 00:00:00,216540,10117,206423
4,ED1007,C007,Hindi,Jharkhand,2020-10-01 00:00:00,234563,13048,221515


In [87]:
def preprocess_print_sales(print_sales_df):
	# Create a copy
	print_sales_df_copy = print_sales_df.copy()

	# Convert Month to datetime
	print_sales_df_copy['Month'] = pd.to_datetime(print_sales_df_copy['Month'], errors='coerce')

	# Extract year , month , month_name and quarter
	print_sales_df_copy['year'] = print_sales_df_copy['Month'].dt.year
	print_sales_df_copy['month_num'] = print_sales_df_copy['Month'].dt.month
	print_sales_df_copy['month_name'] = print_sales_df_copy['Month'].dt.month_name()
	print_sales_df_copy['quarter'] = print_sales_df_copy['Month'].dt.quarter

	return print_sales_df_copy

print_sales_df = preprocess_print_sales(print_sales_df)

In [88]:
print_sales_df.sample(10)

,edition_ID,City_ID,Language,State,Month,Copies Sold,copies_returned,Net_Circulation,year,month_num,month_name,quarter
272,ED1006,C006,english,maharashtra,2024-10-01,303020,9716,293304,2024,10,October,4
382,ED1001,C001,hindi,Uttar pradesh,2020-06-01,195141,12695,182446,2020,6,June,2
342,ED1006,C006,english,maharashtra,2019-12-01,405002,23327,381675,2019,12,December,4
106,ED1003,C003,Hindi,Madhya_Pradesh,2022-02-01,238270,8488,229782,2022,2,February,1
531,ED1002,C002,ENGLISH,Delhi,2023-03-01,317071,21657,295414,2023,3,March,1
594,ED1007,C007,Hindi,Jharkhand,2021-05-01,231707,13514,218193,2021,5,May,2
301,ED1004,C004,Hindi,bihar,2024-03-01,191673,7207,184466,2024,3,March,1
227,ED1001,C001,hindi,Uttar pradesh,2024-07-01,160376,6900,153476,2024,7,July,3
682,ED1003,C003,Hindi,Madhya_Pradesh,2022-10-01,231160,9887,221273,2022,10,October,4
560,ED1006,C006,english,maharashtra,2023-03-01,â‚¹351024,15435,335589,2023,3,March,1


### Storing the cleaned datasets for futher analysis.

In [89]:
# Path of the current directory
current_dir = Path.cwd()

# Move to Bharat-Herald-Analytics directory
main_dir = current_dir.parent

# Define paths for cleaned data
cleaned_data_dir = main_dir / 'data_cleaned'

# Create the directory if it doesn't exist
cleaned_data_dir.mkdir(parents=True, exist_ok=True)

print("Directory for cleaned data is set up at:", cleaned_data_dir)


Directory for cleaned data is set up at: c:\Users\GUMMUDU HEMANTH\Bharat-Herald-Analytics\data_cleaned


In [90]:
city_df.to_excel(cleaned_data_dir / 'cleaned_city_data.xlsx', index=False)
ad_revenue_df.to_csv(cleaned_data_dir / 'cleaned_ad_revenue.csv', index=False)
city_readiness_df.to_csv(cleaned_data_dir / 'cleaned_city_readiness.csv', index=False)
digital_plot_df.to_csv(cleaned_data_dir / 'cleaned_digital_plot.csv', index=False)
print_sales_df.to_excel(cleaned_data_dir / 'cleaned_print_sales.xlsx', index=False)
ad_category_df.to_excel(cleaned_data_dir / 'cleaned_ad_category.xlsx', index=False)
print("Cleaned datasets have been saved successfully.")

Cleaned datasets have been saved successfully.
